In [1]:
import torch.nn as nn
import torch.nn.functional as F
import net_size_utils as nsu

class Net(nn.Module):
    def __init__(self, kernel_size=5):
        super().__init__()
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, 6, kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        linear_input_size = 16 * conv_width * conv_height
        self.fc1 = nn.Linear(linear_input_size, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim

In [2]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,
                                 epochs=epochs, optimizer=optimizer, print_results=False)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, batch_size=32):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net = net_generator()
        optimizer = optim.Adam(net.parameters(), lr=0.001)
        results = train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size=batch_size)
        all_results.append(results)
        net_training.print_final_results(results)
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [3]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]

transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)

#optimizer = optim.Adam(net.parameters(), lr=0.001)

results = cross_validate_net(Net, dataset, repeat=5, epochs=100, batch_size=64)
print_validation_results(results)

Training network 1 ...
Final loss: 1.9140, final accuracy: 62.67 %
	ALBATROSS: 44.44%
	BALD EAGLE: 72.73%
	BARN OWL: 77.14%
	EURASIAN MAGPIE: 65.52%
	FLAMINGO: 69.23%
	MALLARD DUCK: 66.67%
	OSTRICH: 59.26%
	PEACOCK: 72.41%
	PELICAN: 38.71%
	TRUMPTER SWAN: 57.14%
Training network 2 ...
Final loss: 1.8436, final accuracy: 65.07 %
	ALBATROSS: 38.89%
	BALD EAGLE: 57.50%
	BARN OWL: 68.97%
	EURASIAN MAGPIE: 64.10%
	FLAMINGO: 50.00%
	MALLARD DUCK: 85.71%
	OSTRICH: 80.95%
	PEACOCK: 96.88%
	PELICAN: 63.64%
	TRUMPTER SWAN: 57.14%
Training network 3 ...
Final loss: 1.3696, final accuracy: 69.18 %
	ALBATROSS: 56.25%
	BALD EAGLE: 81.48%
	BARN OWL: 57.69%
	EURASIAN MAGPIE: 76.47%
	FLAMINGO: 79.17%
	MALLARD DUCK: 59.09%
	OSTRICH: 69.23%
	PEACOCK: 85.71%
	PELICAN: 58.62%
	TRUMPTER SWAN: 64.86%
Training network 4 ...
Final loss: 1.4331, final accuracy: 71.92 %
	ALBATROSS: 68.42%
	BALD EAGLE: 71.11%
	BARN OWL: 71.43%
	EURASIAN MAGPIE: 87.50%
	FLAMINGO: 64.00%
	MALLARD DUCK: 67.86%
	OSTRICH: 79.31%
	PEAC

In [4]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/net_2conv_2fc_optimizer_adam_001.csv")